<img src="../assets/ittc_logo_full.png" height=150>

# Lecture 4 Practical: Data Cleaning

## In this practical

In this practical you will:

1. Learn how to clean data
2. Search for outliers using automated methods
3. Merge tables both horizontally and vertically
4. Resample time series data onto new frequencies


## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt # Plottling library used for BoxPlots

concrete = pd.read_csv("../data/concrete.csv")
chemical = pd.read_csv("../data/chemicalmanufacturingprocess.csv", skiprows=1)
backblaze = pd.read_parquet("../data/backblaze.parquet")


## 1. Identifying Outliers

### 1.1 Concrete outliers

We will start by using a **box plot** on the `CompressiveStrength` field for the concrete to explore the distribution of values.

A box plot provides a visual guide to the distribution and skew of a data set:

* The lower and upper edges of the box indicate the 1st and 3rd **quartiles** of the data
* The central (orange) line indicates the **median**
* The whiskers indicate 3x the **interquartile range**
* **Outliers** beyond this range are indicated by individual points

In [ ]:
plt.boxplot(concrete["CompressiveStrength"], whis=3, tick_labels=["Compressive Strength"]);

**Observation:** It is clear that there is one outlier value that is far stronger than the general population.

After discussions with engineers who gathered this data, it turns out this value is incorrect.

We need to remove this data point, and to do so we're going to use two methods:

1. Manually identify and remove the outlier
2. An automated method using the interquartile range

#### 1.1 Manual excision

First, identify the row with the outlier value. Use `.query()` to select the row:

In [ ]:
# idx = concrete.query("...").index

Then we can use the method `.drop(rowidx)` to remove the row:

In [ ]:
# concrete.drop(idx)

**Note:** `.drop(()` returns a _new_ DataFrame with the row removed; it will not alter the original DataFrame unless you pass the keyword argument `inplace=True`.

### 1.2 Automated excision

Manually eyeballing outliers is not practical on a large dataset. A better way to do this is to identify outliers in an automated fashion.

Let's use the 3x the interquartile range as a reasonable bound on our data and drop values that lie outside this range.

First, let's calculate the median and interquartile range using `quantile()`:

In [ ]:
# q1, median, q3 = concrete["CompressiveStrength"].quantile([0.25, 0.5, 0.75])

# iqr = ...?

Using these values let's use the `.query()` method to filter the DataFrame to those rows with a CompressiveStength less than a factor 3x the IQR:

In [ ]:
# concretefiltered = concrete.query("(@median - 3 * @iqr) < CompressiveStrength < ...")

Finally, let's visualise our dataset again using the boxplot to confirm the results of our outlier detection:

In [ ]:
# plt.boxplot(concretefiltered["CompressiveStrength"], whis=3, tick_labels=["CompressiveStrength"]);

### 1.3 Chemical Manufacturing Process

The data stored in `chemical` DataFrame also has some outliers associated with its Yield column. In this exercise:

1. Make a boxplot of the Yield
2. Manually identify the outliers and use `.query()` and `.drop()` to excise the data
3. Use the interquartile range to automate the removal of these outliers.

In [ ]:
# Clean your data here

## 2. Merging tables

### 2.1 Vertical concatenation

You receive a new set of concrete data located at `../data/concrete-2.csv`.

**Aim:** combine this table with the original concrete data.

First, load the new data as a DataFrame:

In [ ]:
# concrete2 = pd.read_csv("...")
# display(concrete2)

These tables have an identical set of column, so you can "stack" tables using the function `pd.concat([tbl1, ...])`.

Combine `concrete` and `concrete2` using this function:

In [ ]:
# pd.concat([...])

**Question:** what is the shape of the combined table?

### 2.2 Horizontal merging

First let's note the different models of drive that we have in our data using `.unique()`:

In [ ]:
# backblaze["model"].unique()

Let's load the data about each of these models from the CSV located at `../data/backblaze-models.csv`:

In [ ]:
# models = pd.read_csv(...)

We want to merge these tables: for every row in the original `backblaze` DataFrame, we want to include this additional data about the drive.

We do this using a `pd.merge()`:

* Specify both the "left" and "right" tables to be merged
* Using the keyword parameters `left_on` and `right_on`, specify the column from each table that will be used to match the rows

In [ ]:
# pd.merge(..., ..., left_on="...", right_on="...")

Suppose we're only interested in the operating power from the `models` DataFrame.

Try merging _only_ the `operating_power` column.

(**Hint:** remember we can select a subset of columns with the syntax `df[["col1", "col2"]]` )

In [ ]:
# pd.merge(..., ..., left_on="...", right_on="...")

## 3. Resampling

For this section we return to the Backblaze data.

Each row in this table represents a daily "health check" for every harddrive in operation. Eventually, when a drive fails its final daily health check will set `failure=1`. That is, every entry where `failure=1` represents a drive failing.

**Our goal:** track the number of drives failing over time.

### 3.1 Setting the index

The first step to resampling the backblaze data is to set the index correctly: currently the index is just the row number, but we need the index to be the date.

To make this change, we use the `.set_index()` method:

In [ ]:
backblaze.set_index("date")

You can see that the index column is now set as `date`. Remember that this method returns a new copy of the DataFrame; it does not change the original backblaze DataFrame unless the keyword argument `inplace=True` is set.

### 3.2 Daily failures

Let's say we want to count the number of drive failures on a daily basis. To do this:

1. Set the `date` column as the index
2. Select the `failure` comlumn
2. Resample this column to a daily basis and summing the number of failures.

In [ ]:
# 1. Set the date columnas the index
backblaze_dateindex = backblaze.set_index("date")

# 2. Select the failure column
failures = backblaze_dateindex["failure"]

# 3. Sum the failures for each day
daily = failures.resample("1D").sum()

# OR: Use method chaining to do this as one line:
daily = backblaze.set_index("date")["failure"].resample("1D").sum()

display(daily)


We can now plot these failures over time using `plt.plot()`:

In [ ]:
plt.plot(daily)

### 3.3 Monthly failures

The daily failures are every 'noisy': there's too much variation on a day-to-day basis.

Your goal is to:

1. Resample the data to a **monthly** basis and calculate the total drive failures across this period
2. Create a new plot showing the monthly failure incidence.

**Hint:** You can view all the resamping string codes [here.](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#period-aliases)

In [ ]:
# monthly = ...

## 4. Advanced

Plot the sum operating power for each failed drive over a 7 day period.

You will need to:

* Merge the operating_power column from `models`
* Filter rows to include `failure=1` only
* Resample the data to a '7 day' period
* Sum the operating_power and plot the resulting values

In [ ]:
# ...